In [23]:
using FermiCG
using PyCall
using Plots
using LinearAlgebra
using Printf

## Import PYSCF data

In [24]:
pyscf = pyimport("pyscf");
fcidump = pyimport("pyscf.tools.fcidump");
ctx = fcidump.read("fe2s2");
h = ctx["H1"];
g = ctx["H2"];
ecore = ctx["ECORE"];
g = pyscf.ao2mo.restore("1", g, size(h,2));

In [76]:
# blocking from paper
blocks = [[0,1],[2,3,4,5,6],[7,9,11],[8,10,12],[13,14,15,16,17],[18,19]]
init_fspace = ((2,2),(5,0),(3,3),(3,3),(5,0),(2,2))

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)

6-element Array{Cluster,1}:
 Cluster(1, [1, 2])
 Cluster(2, [3, 4, 5, 6, 7])
 Cluster(3, [8, 10, 12])
 Cluster(4, [9, 11, 13])
 Cluster(5, [14, 15, 16, 17, 18])
 Cluster(6, [19, 20])

In [41]:
# blocking 1
blocks = [  [0,1,2,3,6,7,8],        # L1(sig)+ Fe1(t) + L2(sig)
            [11,12,13,16,17,18,19], # L2(sig)+ Fe2(t) + L3(sig)
            [4,5,9,10,14,15],       # Fe1(e) + Fe2(e) + L2(pi)
         ]

init_fspace = ((7,4), (7,4), (4,4))

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)


3-element Array{Cluster,1}:
 Cluster(1, [1, 2, 3, 4, 7, 8, 9])
 Cluster(2, [12, 13, 14, 17, 18, 19, 20])
 Cluster(3, [5, 6, 10, 11, 15, 16])

In [77]:
ints = InCoreInts(ecore,h,g);

na = 0
nb = 0
for f in init_fspace
    na += f[1]
    nb += f[2]
end
display(na)
display(nb)
# get initial guess for 1RDM
tmp = eigen(ints.h1).vectors;
rdm1a_guess = tmp[:,1:na]*tmp[:,1:na]';
rdm1b_guess = tmp[:,1:nb]*tmp[:,1:nb]';
# eigen(ints.h1).values


20

10

In [79]:
e_cmf, U, Da, Db  = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1a_guess, rdm1b_guess,
                                        max_iter_oo=40, verbose=0, gconv=1e-9, method="bfgs");
ints2 = FermiCG.orbital_rotation(ints,U);


 ooCMF Iter:    1 Total= -115.958971122269 Active= -115.958971122269 G=     5.70e-01
 ooCMF Iter:    2 Total= -116.448055747038 Active= -116.448055747038 G=     2.81e-01
 ooCMF Iter:    3 Total= -116.486525849064 Active= -116.486525849064 G=     1.54e-01
 ooCMF Iter:    4 Total= -116.504043334134 Active= -116.504043334134 G=     7.40e-02
 ooCMF Iter:    5 Total= -116.505837551261 Active= -116.505837551261 G=     2.62e-02
 ooCMF Iter:    6 Total= -116.506350505937 Active= -116.506350505937 G=     1.27e-02
 ooCMF Iter:    7 Total= -116.506659694172 Active= -116.506659694172 G=     8.58e-03
 ooCMF Iter:    8 Total= -116.506638041940 Active= -116.506638041940 G=     3.49e-03
 ooCMF Iter:    9 Total= -116.506628683570 Active= -116.506628683570 G=     1.44e-03
 ooCMF Iter:   10 Total= -116.506636245488 Active= -116.506636245488 G=     5.59e-04
 ooCMF Iter:   11 Total= -116.506631250518 Active= -116.506631250518 G=     2.65e-04
 ooCMF Iter:   12 Total= -116.506632766317 Active= -116.506632766

 * Status: success

 * Candidate solution
    Final objective value:     -1.165066e+02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 2.85e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 7.47e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.74e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.35e-13 ≰ 0.0e+00
    |g(x)|                 = 4.66e-10 ≤ 1.0e-09

 * Work counters
    Seconds run:   2  (vs limit Inf)
    Iterations:    29
    f(x) calls:    87
    ∇f(x) calls:   87


 ooCMF Iter:   30 Total= -116.506633081762 Active= -116.506633081762 G=     2.11e-09
*ooCMF -116.50663308 


In [81]:
e_cmf2, U2, Da2, Db2  = FermiCG.cmf_oo(ints2, clusters, init_fspace, rdm1a_guess, rdm1b_guess,
                                        max_iter_oo=40, verbose=0, gconv=1e-6, method="bfgs");


 ooCMF Iter:    1 Total= -116.506633081762 Active= -116.506633081762 G=     2.85e-02
 ooCMF Iter:    2 Total= -116.507068495076 Active= -116.507068495076 G=     9.04e-03
 ooCMF Iter:    3 Total= -116.507124640709 Active= -116.507124640709 G=     2.19e-03
 ooCMF Iter:    4 Total= -116.507129879271 Active= -116.507129879271 G=     1.16e-03
 ooCMF Iter:    5 Total= -116.507131679987 Active= -116.507131679987 G=     6.54e-04
 ooCMF Iter:    6 Total= -116.507132160106 Active= -116.507132160106 G=     3.05e-04
 ooCMF Iter:    7 Total= -116.507132227701 Active= -116.507132227701 G=     1.45e-04
 ooCMF Iter:    8 Total= -116.507132246909 Active= -116.507132246909 G=     6.64e-05
 ooCMF Iter:    9 Total= -116.507132249885 Active= -116.507132249885 G=     3.19e-05
 ooCMF Iter:   10 Total= -116.507132250670 Active= -116.507132250670 G=     6.81e-06


 * Status: success

 * Candidate solution
    Final objective value:     -1.165071e+02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 3.10e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 3.02e-04 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.61e-11 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.10e-13 ≰ 0.0e+00
    |g(x)|                 = 8.95e-07 ≤ 1.0e-06

 * Work counters
    Seconds run:   188  (vs limit Inf)
    Iterations:    10
    f(x) calls:    31
    ∇f(x) calls:   31


 ooCMF Iter:   11 Total= -116.507132250706 Active= -116.507132250706 G=     2.74e-06
*ooCMF -116.50713225 
